# Step-by-step YAO's Garbled Circuit

In [ ]:
circuit = {
  "name": "smart",
  "circuits": [
    {
      "id": "Smart",
      "alice": [1, 2],
      "bob": [3, 4],
      "out": [7],
      "gates": [
        {"id": 5, "type": "AND", "in": [1, 3]},
        {"id": 6, "type": "XOR", "in": [2, 4]},
        {"id": 7, "type": "OR", "in": [5, 6]}
      ]
    }
  ]
}

## Garbling Scheme

### Truth table

In [ ]:
def generate_truth_table():
    truth_table = {
        "Gate 5 (AND)": [],
        "Gate 6 (XOR)": [],
        "Gate 7 (OR)": []
    }

    # Compute truth table for Gate 5 (AND)
    for a1 in [0, 1]:
        for b1 in [0, 1]:
            gate_5 = a1 and b1
            truth_table["Gate 5 (AND)"].append(((a1, b1), gate_5))

    # Compute truth table for Gate 6 (XOR)
    for a2 in [0, 1]:
        for b2 in [0, 1]:
            gate_6 = a2 ^ b2
            truth_table["Gate 6 (XOR)"].append(((a2, b2), gate_6))

    # Compute truth table for Gate 7 (OR) based on previous gate outputs
    for gate_5 in [0, 1]:
        for gate_6 in [0, 1]:
            gate_7 = gate_5 or gate_6
            truth_table["Gate 7 (OR)"].append(((gate_5, gate_6), gate_7))

    return truth_table

truth_table = generate_truth_table()

# Print the truth table for each gate
for gate, table in truth_table.items():
    print(f"{gate}:")
    for inputs, output in table:
        print(f"  Inputs: {inputs} -> Output: {output}")


Gate 5 (AND):
  Inputs: (0, 0) -> Output: 0
  Inputs: (0, 1) -> Output: 0
  Inputs: (1, 0) -> Output: 0
  Inputs: (1, 1) -> Output: 1
Gate 6 (XOR):
  Inputs: (0, 0) -> Output: 0
  Inputs: (0, 1) -> Output: 1
  Inputs: (1, 0) -> Output: 1
  Inputs: (1, 1) -> Output: 0
Gate 7 (OR):
  Inputs: (0, 0) -> Output: 0
  Inputs: (0, 1) -> Output: 1
  Inputs: (1, 0) -> Output: 1
  Inputs: (1, 1) -> Output: 1


### Set labels for each wire

In [ ]:
import random
import string

def generate_random_label(k=4):
    """Generate a random label of k bits."""
    return ''.join(random.choices(string.ascii_letters + string.digits, k=k))

def generate_truth_table_with_labels(k=4):
    # Generate random labels for each wire in the circuit
    labels = {
        "A1": (generate_random_label(k), generate_random_label(k)),
        "A2": (generate_random_label(k), generate_random_label(k)),
        "B1": (generate_random_label(k), generate_random_label(k)),
        "B2": (generate_random_label(k), generate_random_label(k)),
        "G5": (generate_random_label(k), generate_random_label(k)),
        "G6": (generate_random_label(k), generate_random_label(k)),
        "G7": (generate_random_label(k), generate_random_label(k)),
    }

    truth_table = {
        "Gate 5 (AND)": [],
        "Gate 6 (XOR)": [],
        "Gate 7 (OR)": []
    }

    for a1 in [0, 1]:
        for b1 in [0, 1]:
            gate_5 = a1 and b1
            truth_table["Gate 5 (AND)"].append((
                (labels["A1"][a1], labels["B1"][b1]),
                labels["G5"][gate_5]
            ))

    for a2 in [0, 1]:
        for b2 in [0, 1]:
            gate_6 = a2 ^ b2
            truth_table["Gate 6 (XOR)"].append((
                (labels["A2"][a2], labels["B2"][b2]),
                labels["G6"][gate_6]
            ))

    for gate_5 in [0, 1]:
        for gate_6 in [0, 1]:
            gate_7 = gate_5 or gate_6
            truth_table["Gate 7 (OR)"].append((
                (labels["G5"][gate_5], labels["G6"][gate_6]),
                labels["G7"][gate_7]
            ))

    return truth_table, labels

truth_table, labels = generate_truth_table_with_labels()

for gate, table in truth_table.items():
    print(f"{gate}:")
    for inputs, output in table:
        print(f"  Inputs: {inputs} -> Output: {output}")

# Print the labels
print("\nLabels:")
for wire, (label0, label1) in labels.items():
    print(f"{wire}: 0 -> {label0}, 1 -> {label1}")


Gate 5 (AND):
  Inputs: ('Htrc', '1aMd') -> Output: rLXd
  Inputs: ('Htrc', 'IMwq') -> Output: rLXd
  Inputs: ('hEFW', '1aMd') -> Output: rLXd
  Inputs: ('hEFW', 'IMwq') -> Output: i1uY
Gate 6 (XOR):
  Inputs: ('fDkE', 'Uhja') -> Output: lyvU
  Inputs: ('fDkE', 'Gek4') -> Output: MMDk
  Inputs: ('QUd6', 'Uhja') -> Output: MMDk
  Inputs: ('QUd6', 'Gek4') -> Output: lyvU
Gate 7 (OR):
  Inputs: ('rLXd', 'lyvU') -> Output: CNGz
  Inputs: ('rLXd', 'MMDk') -> Output: FJwk
  Inputs: ('i1uY', 'lyvU') -> Output: FJwk
  Inputs: ('i1uY', 'MMDk') -> Output: FJwk

Labels:
A1: 0 -> Htrc, 1 -> hEFW
A2: 0 -> fDkE, 1 -> QUd6
B1: 0 -> 1aMd, 1 -> IMwq
B2: 0 -> Uhja, 1 -> Gek4
G5: 0 -> rLXd, 1 -> i1uY
G6: 0 -> lyvU, 1 -> MMDk
G7: 0 -> CNGz, 1 -> FJwk


### Encrypted table

In [ ]:
import random
import string

def generate_random_label(k=4):
    """Generate a random label of k bits."""
    return ''.join(random.choices(string.ascii_letters + string.digits, k=k))

def encrypt(value, key):
    """Encrypt the value using the key."""
    # For simplicity, let's just concatenate the value and key
    return value + key

def generate_truth_table_with_labels_encrypted(k=4):
    # Generate random labels for each wire in the circuit
    labels = {
        "A1": (generate_random_label(k), generate_random_label(k)),
        "A2": (generate_random_label(k), generate_random_label(k)),
        "B1": (generate_random_label(k), generate_random_label(k)),
        "B2": (generate_random_label(k), generate_random_label(k)),
        "G5": (generate_random_label(k), generate_random_label(k)),
        "G6": (generate_random_label(k), generate_random_label(k)),
        "G7": (generate_random_label(k), generate_random_label(k)),
    }

    truth_table = {
        "Gate 5 (AND)": [],
        "Gate 6 (XOR)": [],
        "Gate 7 (OR)": []
    }

    for a1 in [0, 1]:
        for b1 in [0, 1]:
            gate_5 = a1 and b1
            encrypted_output = encrypt(labels["G5"][gate_5], labels["A1"][a1] + labels["B1"][b1])
            truth_table["Gate 5 (AND)"].append((labels["A1"][a1], labels["B1"][b1], encrypted_output))

    for a2 in [0, 1]:
        for b2 in [0, 1]:
            gate_6 = a2 ^ b2
            encrypted_output = encrypt(labels["G6"][gate_6], labels["A2"][a2] + labels["B2"][b2])
            truth_table["Gate 6 (XOR)"].append((labels["A2"][a2], labels["B2"][b2], encrypted_output))

    for gate_5 in [0, 1]:
        for gate_6 in [0, 1]:
            gate_7 = gate_5 or gate_6
            encrypted_output = encrypt(labels["G7"][gate_7], labels["G5"][gate_5] + labels["G6"][gate_6])
            truth_table["Gate 7 (OR)"].append((labels["G5"][gate_5], labels["G6"][gate_6], encrypted_output))

    return truth_table, labels

truth_table, labels = generate_truth_table_with_labels_encrypted()

# Print the truth table for each gate
for gate, table in truth_table.items():
    print(f"{gate}:")
    for input_A, input_B, output in table:
        print(f"  Inputs: {input_A}, {input_B} -> Encrypted Output: {output}")

# Print the labels
print("\nLabels:")
for wire, (label0, label1) in labels.items():
    print(f"{wire}: 0 -> {label0}, 1 -> {label1}")


Gate 5 (AND):
  Inputs: 8C3L, wc0X -> Encrypted Output: f7rM8C3Lwc0X
  Inputs: 8C3L, RTPy -> Encrypted Output: f7rM8C3LRTPy
  Inputs: 3Zfe, wc0X -> Encrypted Output: f7rM3Zfewc0X
  Inputs: 3Zfe, RTPy -> Encrypted Output: njR83ZfeRTPy
Gate 6 (XOR):
  Inputs: pdfz, bRDP -> Encrypted Output: v9vPpdfzbRDP
  Inputs: pdfz, LVo5 -> Encrypted Output: YqjUpdfzLVo5
  Inputs: DDXh, bRDP -> Encrypted Output: YqjUDDXhbRDP
  Inputs: DDXh, LVo5 -> Encrypted Output: v9vPDDXhLVo5
Gate 7 (OR):
  Inputs: f7rM, v9vP -> Encrypted Output: g4DVf7rMv9vP
  Inputs: f7rM, YqjU -> Encrypted Output: piwDf7rMYqjU
  Inputs: njR8, v9vP -> Encrypted Output: piwDnjR8v9vP
  Inputs: njR8, YqjU -> Encrypted Output: piwDnjR8YqjU

Labels:
A1: 0 -> 8C3L, 1 -> 3Zfe
A2: 0 -> pdfz, 1 -> DDXh
B1: 0 -> wc0X, 1 -> RTPy
B2: 0 -> bRDP, 1 -> LVo5
G5: 0 -> f7rM, 1 -> njR8
G6: 0 -> v9vP, 1 -> YqjU
G7: 0 -> g4DV, 1 -> piwD


### Garbled table

In [ ]:
# Function to generate a random permutation
def random_permutation(table):
    permuted_table = table[:]  # Make a copy of the table
    random.shuffle(permuted_table)  # Shuffle the copy
    return permuted_table

truth_table, labels = generate_truth_table_with_labels_encrypted()

# Permute the truth table rows
for gate, table in truth_table.items():
    truth_table[gate] = random_permutation(table)

for gate, table in truth_table.items():
    print(f"{gate}:")
    for input_A, input_B, output in table:
        print(f"  Inputs: A={input_A}, B={input_B} -> Encrypted Output: {output}")

print("\nLabels:")
for wire, (label0, label1) in labels.items():
    print(f"{wire}: 0 -> {label0}, 1 -> {label1}")


Gate 5 (AND):
  Inputs: A=UjfN, B=t56m -> Encrypted Output: zb2dUjfNt56m
  Inputs: A=UjfN, B=7wVe -> Encrypted Output: q3O8UjfN7wVe
  Inputs: A=5MKU, B=7wVe -> Encrypted Output: zb2d5MKU7wVe
  Inputs: A=5MKU, B=t56m -> Encrypted Output: zb2d5MKUt56m
Gate 6 (XOR):
  Inputs: A=lVUb, B=EiqP -> Encrypted Output: cGRglVUbEiqP
  Inputs: A=lVUb, B=2Owm -> Encrypted Output: G27alVUb2Owm
  Inputs: A=odlg, B=EiqP -> Encrypted Output: G27aodlgEiqP
  Inputs: A=odlg, B=2Owm -> Encrypted Output: cGRgodlg2Owm
Gate 7 (OR):
  Inputs: A=q3O8, B=G27a -> Encrypted Output: xei0q3O8G27a
  Inputs: A=zb2d, B=G27a -> Encrypted Output: bL2Ozb2dG27a
  Inputs: A=q3O8, B=cGRg -> Encrypted Output: xei0q3O8cGRg
  Inputs: A=zb2d, B=cGRg -> Encrypted Output: xei0zb2dcGRg

Labels:
A1: 0 -> 5MKU, 1 -> UjfN
A2: 0 -> lVUb, 1 -> odlg
B1: 0 -> t56m, 1 -> 7wVe
B2: 0 -> 2Owm, 1 -> EiqP
G5: 0 -> zb2d, 1 -> q3O8
G6: 0 -> G27a, 1 -> cGRg
G7: 0 -> bL2O, 1 -> xei0


## 1-out-of-2 Oblivious Transfers

In [ ]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

def generate_rsa_key_pair():
    # Generate RSA key pair
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    public_key = private_key.public_key()

    # Serialize keys to PEM format
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    return private_pem, public_pem

def rsa_encrypt(public_key_pem, plaintext):
    # Load public key
    public_key = serialization.load_pem_public_key(public_key_pem)

    # Generate padding and encrypt
    ciphertext = public_key.encrypt(
        plaintext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    return ciphertext

def rsa_decrypt(private_key_pem, ciphertext):
    # Load private key
    private_key = serialization.load_pem_private_key(
        private_key_pem,
        password=None
    )

    # Decrypt and remove padding
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    return plaintext


def oblivious_transfer(message0, message1, chosen_bit):

    def pk_sampling(chosen_bit):
        private_key_pem0, public_key_pem0 = generate_rsa_key_pair()
        private_key_pem1, public_key_pem1 = generate_rsa_key_pair()
        if chosen_bit == 0:
            return public_key_pem0, public_key_pem1, private_key_pem0
        return public_key_pem0, public_key_pem1, private_key_pem1

    def encrypt_messages(message0, message1, key0, key1):
        return rsa_encrypt(key0, message0), rsa_encrypt(key1, message1)

    def decrypt_message(enc_massage0, enc_message1, private_key):
        try:
            message = rsa_decrypt(private_key, enc_massage0)
        except:
            message = rsa_decrypt(private_key, enc_message1)
        return message

    # receiver generate keys
    pk0, pk1, sk = pk_sampling(chosen_bit)
    # sender receive public keys and encrypt both of its messages
    enc_msg1, enc_msg2 = encrypt_messages(message0, message1, pk0, pk1)
    # receiver receive both messages but only decrypt the one correspond to its chosen bit
    chosen_msg = decrypt_message(enc_msg1, enc_msg2, sk)
    return chosen_msg



sender_message0 = b"Message 0"
sender_message1 = b"Message 1"

receiver_chosen_bit = 0
receiver_chosen_message = oblivious_transfer(sender_message0, sender_message1, receiver_chosen_bit)
print("Receiver's choice:", receiver_chosen_bit)
print("Decrypted text:", receiver_chosen_message.decode())


Receiver's choice: 0
Decrypted text: Message 0
